# Gradio UI Intro

In [2]:
# imports
import os
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr

In [11]:
def shout(txt):
    print(f"The text which has been shouted is---> {txt}")
    return txt.upper()

In [4]:
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [8]:
# removing the flag from our ui
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [12]:
# Create a public link so, that we can share with someone else
view = gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://618a80031d67051c89.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The text which has been shouted is---> hlo
The text which has been shouted is---> my name is anuj
The text which has been shouted is---> who are you


### Creating a chat bot with gemini-2.0-flash

In [21]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# getting api keys
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")
gemini_MODEL = 'gemini-2.0-flash'
gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

# Creating our ai client
gemini_client = OpenAI(
    api_key=gemini_api_key,
    base_url=gemini_base_url
)

# basic func to get the res from gemini
def gemini_res(query):
    if query.lower() == 'quit':
        return "Goodbye"

    context = [
        {'role':'system', 'content':"You are a very helpfull assistant"},
        {'role':'user', 'content':query}
    ]
    
    res = gemini_client.chat.completions.create(
        model=gemini_MODEL,
        messages=context
    )

    return res.choices[0].message.content

In [22]:
gemini_res('hi')

'Hi there! How can I help you today?\n'

In [26]:
# creating basic ui for above gemini model
view = gr.Interface(
    fn=gemini_res,
    inputs=[gr.Textbox(label="Your Message", lines=6)],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode='never'
).launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


In [30]:
# Now adding a stream content in our gemini model
import os
from dotenv import load_dotenv
from openai import OpenAI

# getting api keys
load_dotenv()
gemini_api_key = os.getenv("GOOGLE_API_KEY")
gemini_MODEL = 'gemini-2.0-flash'
gemini_base_url="https://generativelanguage.googleapis.com/v1beta/openai/"

# Creating our ai client
gemini_client = OpenAI(
    api_key=gemini_api_key,
    base_url=gemini_base_url
)

# basic func to get the res from gemini
def gemini_res_stream(query):
    if query.lower() == 'quit':
        return "Goodbye"

    context = [
        {'role':'system', 'content':"You are a very helpfull assistant, who response in the markdown"},
        {'role':'user', 'content':query}
    ]
    
    stream = gemini_client.chat.completions.create(
        model=gemini_MODEL,
        messages=context,
        stream=True
    )

    result=""
    for chunks in stream:
        result = result + chunks.choices[0].delta.content or ""
        yield result


In [31]:
view = gr.Interface(
    fn=gemini_res_stream,
    inputs=[gr.Textbox(label="Your Message", lines=8)],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never"
).launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


In [32]:
# Creating another model so that we can give choice to the user which model they want to use
# Now adding a stream content in our gemini model
import os
from dotenv import load_dotenv
from openai import OpenAI

# getting api keys
load_dotenv()
supernova_api_key = os.getenv("SAMBANOVA_API_KEY")
supernova_MODEL = 'Llama-4-Maverick-17B-128E-Instruct'
supernova_base_url="https://api.sambanova.ai/v1"

# Creating our ai client
supernova_client = OpenAI(
    api_key=supernova_api_key,
    base_url=supernova_base_url
)

# basic func to get the res from gemini
def sambanova_res_stream(query):
    if query.lower() == 'quit':
        return "Goodbye"

    context = [
        {'role':'system', 'content':"You are a very helpfull assistant, who respond in the markdown format."},
        {'role':'user', 'content':query}
    ]
    
    stream = supernova_client.chat.completions.create(
        model=supernova_MODEL,
        messages=context,
        stream=True
    )

    result=""
    for chunks in stream:
        result = result + chunks.choices[0].delta.content or ""
        yield result


In [34]:
def choose_model(query, model):
    if model=="gemini":
        result = gemini_res_stream(query)
    elif model=="sambanova":
        result = sambanova_res_stream(query)
    else:
        raise ValueError("Unknown model was selected")
    yield from result

In [35]:
view = gr.Interface(
    fn=choose_model,
    inputs=[gr.Textbox(label="Your Message", lines=8), gr.Dropdown(["gemini", "sambanova"],label="Choose Model", value='gemini')],
    outputs=[gr.Markdown(label="Response")],
    flagging_mode="never"
).launch(share=True)

* Running on local URL:  http://127.0.0.1:7872
* Running on public URL: https://cf70278c04f6ed6428.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
